In [1]:
using Pkg: @pkg_str

pkg"activate .."
pkg"status"

Activating environment at `~/JuliaEnvs/Arborist/Project.toml`
Project Arborist v0.1.0
    Status `~/JuliaEnvs/Arborist/Project.toml`
  [6e4b80f9] + BenchmarkTools v0.4.2
  [da1fd8a2] + CodeTracking v0.5.7
  [1914dd2f] + MacroTools v0.5.1
  [295af30f] + Revise v2.1.6
  [b77e0a4c] + InteractiveUtils 
    Status `~/JuliaEnvs/Arborist/Manifest.toml`
  [6e4b80f9] + BenchmarkTools v0.4.2
  [da1fd8a2] + CodeTracking v0.5.7
  [1914dd2f] + MacroTools v0.5.1
  [295af30f] + Revise v2.1.6
  [b77e0a4c] + InteractiveUtils 


In [2]:
using Arborist
using Arborist: graft, get_ast, recurse!
using Test
using BenchmarkTools
using MacroTools

┌ Info: Recompiling stale cache file /Users/oxinabox/.julia/compiled/v1.3/Arborist/OXQd1.ji for Arborist [a56570ba-ce5c-531d-8734-586fc97c4b8d]
└ @ Base loading.jl:1240


In [3]:
includet("../test/demo.jl")

In [6]:
function translate_strings_grafter(reflection)
    ast = reflection.ast
    
    MacroTools.postwalk(ast) do expr
        if isexpr(expr, :call)
            return recurse!(expr, reflection)
        elseif isexpr(expr, String) ## String literal
            # Stub for actually translating the text
            return uppercase(expr)
        else
            return expr
        end
    end
end
Arborist.redeclare_graft()

graft (generic function with 3 methods)

In [8]:
@time graft(translate_strings_grafter, string_demo1)

  0.000010 seconds (5 allocations: 192 bytes)


"OUTERINNER"

In [21]:
@time graft(translate_strings_grafter, badcake2)

()
*** badcake2() in Main at /Users/oxinabox/JuliaEnvs/Arborist/test/demo.jl:28
quote
    y = Main.graft(translate_strings_grafter, Main.sum, [1 for ii = Main.graft(translate_strings_grafter, Main.:(:), 1, 3)])
    return y
end

()
*** badcake2() in Main at /Users/oxinabox/JuliaEnvs/Arborist/test/demo.jl:28
quote
    y = Main.graft(translate_strings_grafter, Main.sum, [1 for ii = Main.graft(translate_strings_grafter, Main.:(:), 1, 3)])
    return y
end



ErrorException: generated function body is not pure. this likely means it contains a closure or comprehension.

In [22]:
@inline function fma_grafter(reflection)
    ast = reflection.ast
    MacroTools.prewalk(ast) do expr
        if (
            @capture(expr, x_ * y_ + z_) ||
            @capture(expr, z_ + x_ * y_)        
            )
            
            return :(fma($x, $y, $z)) 
        elseif isexpr(expr, :call)
            return Expr(:call,
                :graft, fma_grafter, expr.args...
            )
        end
        return expr
    end
end

Arborist.redeclare_graft()

graft (generic function with 3 methods)

In [20]:
bigmat = 100000rand((1000,1000))

@time graft(fma_grafter, fma_demo2, bigmat)

(Int64,)
*** fma_demo2(a) in Main at /Users/oxinabox/JuliaEnvs/Arborist/test/demo.jl:43
quote
    (a,) = call_args
    Main.fma(Main.graft(fma_grafter, *, 8, a), a, sqrt.(a))
end

(Int64, Int64)
*** *(x::T, y::T) where T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8} in Base at int.jl:54
  0.131535 seconds (165.59 k allocations: 8.442 MiB, 4.15% gc time)


┌ Warning: skipping *(x::T, y::T) where T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8} in Base at int.jl:54 to avoid parsing too much code
└ @ Revise /Users/oxinabox/.julia/packages/Revise/agmgx/src/Revise.jl:813


8.000000000001e16

In [14]:
@time fma_demo2(bigmat)

  0.000003 seconds (5 allocations: 176 bytes)


8.000000000001e16

In [20]:
@generated function foo()
    xs = (a=2, b=2)
    :(1+1)
end
foo()

2